# Donor Analytics Workflow - Comprehensive Analysis

This notebook provides a complete workflow for analyzing donor data, including:
1. Data Quality Assessment
2. Geospatial Analysis
3. Advanced Analytics
4. Machine Learning Pipeline
5. Visualization Functions
6. Cloud Configuration
7. Streamlit Dashboard Integration

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd
import folium
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from datetime import datetime, timedelta
import great_expectations as ge
import streamlit as st
from pathlib import Path

# Set plotting styles
plt.style.use('seaborn')
sns.set_palette("husl")

# 1. Data Quality Assessment

First, we'll load the data and perform comprehensive quality checks using pandas-profiling and Great Expectations.

In [ ]:
# Load the data
data_dir = Path("../data")
donors = pd.read_csv(data_dir / "raw/donors.csv")
donations = pd.read_csv(data_dir / "raw/donations.csv")
campaigns = pd.read_csv(data_dir / "raw/campaigns.csv")
events = pd.read_csv(data_dir / "raw/engagement_events.csv")
wealth = pd.read_csv(data_dir / "raw/wealth_external.csv")

# Create Great Expectations DataContext
context = ge.data_context.DataContext()

# Create expectations for donors dataset
donor_suite = ge.dataset.PandasDataset(donors)
donor_suite.expect_column_values_to_not_be_null("donor_id")
donor_suite.expect_column_values_to_be_unique("donor_id")
donor_suite.expect_column_values_to_be_between("birth_year", 1900, datetime.now().year)
donor_suite.expect_column_values_to_match_regex("email", r"[^@]+@[^@]+\.[^@]+")
donor_suite.expect_column_values_to_be_in_set("state", ['NY', 'NJ', 'CT', 'PA', 'MA'])  # Adjust states as needed

# Generate validation report
validation_result = donor_suite.validate()
print(f"Data Quality Score: {validation_result.success_percent:.2f}%")

# Display detailed issues
issues = []
for result in validation_result.results:
    if not result.success:
        issues.append({
            'expectation': result.expectation_config.expectation_type,
            'column': result.expectation_config.kwargs.get('column'),
            'success_rate': result.success_percent
        })
pd.DataFrame(issues)

# 2. Geospatial Analysis

Now we'll analyze the geographical distribution of donors and donations using GeoPandas and Folium.

In [ ]:
# Load US states shapefile
states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
states = states[states['continent'] == 'North America']

# Aggregate donor data by state
state_stats = donors.groupby('state').agg({
    'donor_id': 'count',
    'wealth_index': 'mean',
    'engagement_index': 'mean'
}).reset_index()

# Create choropleth map
fig = go.Figure()

fig.add_trace(go.Choropleth(
    locations=state_stats['state'],
    z=state_stats['donor_id'],
    locationmode='USA-states',
    colorscale='Viridis',
    colorbar_title="Number of Donors"
))

fig.update_layout(
    title='Donor Distribution by State',
    geo_scope='usa',
)

fig.show()

# Create interactive Folium map
donor_map = folium.Map(location=[40.7128, -74.0060], zoom_start=4)

# Add donor clusters
for idx, row in donors.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        popup=f"Donor ID: {row['donor_id']}<br>Wealth Index: {row['wealth_index']:.2f}",
        color='red',
        fill=True
    ).add_to(donor_map)

donor_map

# 3. Advanced Analytics

Let's perform advanced statistical analysis and feature engineering.

In [ ]:
# Calculate RFM scores
def calculate_rfm(donations_df, analysis_date=None):
    if analysis_date is None:
        analysis_date = datetime.now()
    
    rfm = donations.groupby('donor_id').agg({
        'donation_date': lambda x: (analysis_date - pd.to_datetime(x.max())).days,  # Recency
        'donation_id': 'count',  # Frequency
        'amount': 'sum'  # Monetary
    }).reset_index()
    
    rfm.columns = ['donor_id', 'recency_days', 'frequency', 'monetary']
    
    # Score from 1-5 (5 being best)
    rfm['r_score'] = pd.qcut(rfm['recency_days'], q=5, labels=[5,4,3,2,1])
    rfm['f_score'] = pd.qcut(rfm['frequency'], q=5, labels=[1,2,3,4,5])
    rfm['m_score'] = pd.qcut(rfm['monetary'], q=5, labels=[1,2,3,4,5])
    
    rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)
    
    return rfm

# Calculate RFM scores
rfm = calculate_rfm(donations)

# Visualize RFM segments
fig = make_subplots(rows=2, cols=2)

# RFM Score Distribution
rfm_dist = rfm['rfm_score'].value_counts()[:10]
fig.add_trace(
    go.Bar(x=rfm_dist.index, y=rfm_dist.values, name='RFM Segments'),
    row=1, col=1
)

# Monetary vs Frequency
fig.add_trace(
    go.Scatter(x=rfm['frequency'], y=rfm['monetary'], mode='markers',
               marker=dict(color=rfm['recency_days'], colorscale='Viridis'),
               name='M vs F'),
    row=1, col=2
)

# Recency Distribution
fig.add_trace(
    go.Histogram(x=rfm['recency_days'], name='Recency Distribution'),
    row=2, col=1
)

# Average Amount by Frequency
avg_by_freq = donations.groupby('donor_id').agg({
    'donation_id': 'count',
    'amount': 'mean'
}).reset_index()

fig.add_trace(
    go.Box(y=avg_by_freq['amount'], x=pd.qcut(avg_by_freq['donation_id'], 5).astype(str),
           name='Amount by Frequency Quintile'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="RFM Analysis Dashboard")
fig.show()

# 4. Machine Learning Pipeline

Now we'll create a machine learning pipeline for donor propensity scoring and clustering.

In [ ]:
# Prepare features for ML
def prepare_ml_features(donors_df, rfm_df, events_df, wealth_df):
    features = donors_df.merge(rfm_df[['donor_id', 'recency_days', 'frequency', 'monetary']], 
                             on='donor_id', how='left')
    features = features.merge(events_df, on='donor_id', how='left')
    features = features.merge(wealth_df, on='donor_id', how='left')
    
    # Fill missing values
    features = features.fillna({
        'events_attended': 0,
        'volunteer_hours': 0,
        'wealth_score_ext': features['wealth_score_ext'].median()
    })
    
    return features

# Prepare data
features_df = prepare_ml_features(donors, rfm, events, wealth)

# Define features for clustering
cluster_features = [
    'monetary', 'frequency', 'recency_days',
    'events_attended', 'volunteer_hours', 'wealth_score_ext'
]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_df[cluster_features])

# Perform k-means clustering
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
features_df['cluster'] = kmeans.fit_predict(X_scaled)

# Analyze clusters
cluster_summary = features_df.groupby('cluster').agg({
    'monetary': 'mean',
    'frequency': 'mean',
    'recency_days': 'mean',
    'events_attended': 'mean',
    'volunteer_hours': 'mean',
    'wealth_score_ext': 'mean',
    'donor_id': 'count'
}).round(2)

# Visualize clusters
fig = px.scatter_3d(
    features_df,
    x='monetary',
    y='frequency',
    z='recency_days',
    color='cluster',
    size='wealth_score_ext',
    hover_data=['donor_id', 'events_attended'],
    title='Donor Clusters (3D View)'
)
fig.show()

# Train XGBoost model for propensity scoring
def create_target(row):
    """Define target variable based on RFM values"""
    if (row['recency_days'] < 120 or 
        row['frequency'] >= 3 or 
        row['monetary'] >= features_df['monetary'].median()):
        return 1
    return 0

features_df['target'] = features_df.apply(create_target, axis=1)

# Prepare features for XGBoost
X = features_df[cluster_features]
y = features_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = xgb.XGBClassifier(
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance plot
feature_importance = pd.DataFrame({
    'feature': cluster_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

px.bar(
    feature_importance,
    x='feature',
    y='importance',
    title='Feature Importance in Propensity Model'
).show()

# 5. Cloud Configuration

Set up configuration templates for different cloud providers.

In [ ]:
# Cloud configuration templates
cloud_configs = {
    'aws': {
        'storage': {
            'bucket': 'donor-analytics-data',
            'region': 'us-east-1',
            'access_key': '${AWS_ACCESS_KEY}',
            'secret_key': '${AWS_SECRET_KEY}'
        },
        'compute': {
            'emr_cluster': {
                'name': 'donor-analytics-cluster',
                'instance_type': 'm5.xlarge',
                'instance_count': 3,
                'applications': ['Spark', 'Hadoop']
            }
        },
        'warehouse': {
            'snowflake': {
                'account': '${SNOWFLAKE_ACCOUNT}',
                'username': '${SNOWFLAKE_USER}',
                'password': '${SNOWFLAKE_PASSWORD}',
                'warehouse': 'DONOR_ANALYTICS_WH',
                'database': 'DONOR_ANALYTICS_DB'
            }
        }
    },
    'azure': {
        'storage': {
            'account_name': 'donoranalytics',
            'container': 'donor-data',
            'connection_string': '${AZURE_STORAGE_CONNECTION_STRING}'
        },
        'compute': {
            'synapse': {
                'workspace_name': 'donor-analytics-synapse',
                'spark_pool': 'DonorAnalyticsPool',
                'node_size': 'Medium',
                'node_count': 3
            }
        },
        'warehouse': {
            'synapse_sql': {
                'database': 'DonorAnalyticsDB',
                'server': '${AZURE_SQL_SERVER}',
                'user': '${AZURE_SQL_USER}',
                'password': '${AZURE_SQL_PASSWORD}'
            }
        }
    }
}

# Save configurations
import yaml
import json

# Save as YAML
with open('../config/cloud_config.yml', 'w') as f:
    yaml.dump(cloud_configs, f)

# Save as JSON
with open('../config/cloud_config.json', 'w') as f:
    json.dump(cloud_configs, f, indent=2)

# 6. Streamlit Dashboard Components

Create reusable components for the Streamlit dashboard.

In [ ]:
# Define Streamlit dashboard components
class DonorDashboard:
    def __init__(self, features_df, model):
        self.features_df = features_df
        self.model = model
        
    def donor_search(self):
        """Create donor search component"""
        st.sidebar.header("🔍 Donor Search")
        search_term = st.sidebar.text_input("Search by name or ID")
        
        if search_term:
            results = self.features_df[
                self.features_df['donor_id'].astype(str).str.contains(search_term) |
                self.features_df['first_name'].str.contains(search_term, case=False) |
                self.features_df['last_name'].str.contains(search_term, case=False)
            ]
            return results
        return None
    
    def donor_profile(self, donor_id):
        """Create donor profile component"""
        donor = self.features_df[self.features_df['donor_id'] == donor_id].iloc[0]
        
        st.header(f"Donor Profile: {donor['first_name']} {donor['last_name']}")
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.metric("Total Giving", f"${donor['monetary']:,.2f}")
        with col2:
            st.metric("Frequency", f"{donor['frequency']} gifts")
        with col3:
            st.metric("Last Gift", f"{donor['recency_days']} days ago")
            
        # Propensity score
        features = donor[self.model.feature_names_in_]
        propensity = self.model.predict_proba([features])[0][1]
        
        st.progress(propensity)
        st.caption(f"Propensity to Give: {propensity:.1%}")
        
        return donor
    
    def geographic_view(self):
        """Create geographic visualization component"""
        st.header("📍 Geographic Distribution")
        
        # Create choropleth
        state_summary = self.features_df.groupby('state').agg({
            'monetary': 'sum',
            'donor_id': 'count'
        }).reset_index()
        
        fig = go.Figure(data=go.Choropleth(
            locations=state_summary['state'],
            z=state_summary['monetary'],
            locationmode='USA-states',
            colorscale='Viridis',
            colorbar_title="Total Donations ($)"
        ))
        
        fig.update_layout(
            title='Donation Distribution by State',
            geo_scope='usa',
        )
        
        st.plotly_chart(fig)
    
    def rfm_analysis(self):
        """Create RFM analysis component"""
        st.header("📊 RFM Analysis")
        
        # RFM Distribution
        fig = make_subplots(rows=1, cols=3)
        
        fig.add_trace(
            go.Histogram(x=self.features_df['recency_days'], name="Recency"),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Histogram(x=self.features_df['frequency'], name="Frequency"),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Histogram(x=self.features_df['monetary'], name="Monetary"),
            row=1, col=3
        )
        
        fig.update_layout(height=400, title_text="RFM Distributions")
        st.plotly_chart(fig)
    
    def campaign_simulator(self):
        """Create campaign simulator component"""
        st.header("🎯 Campaign Simulator")
        
        col1, col2 = st.columns(2)
        
        with col1:
            min_propensity = st.slider(
                "Minimum Propensity Score",
                min_value=0.0,
                max_value=1.0,
                value=0.5
            )
        
        with col2:
            min_amount = st.slider(
                "Minimum Previous Gift",
                min_value=0,
                max_value=int(self.features_df['monetary'].max()),
                value=1000
            )
        
        # Filter prospects
        prospects = self.features_df[
            (self.model.predict_proba(self.features_df[self.model.feature_names_in_])[:, 1] >= min_propensity) &
            (self.features_df['monetary'] >= min_amount)
        ]
        
        st.metric("Target List Size", len(prospects))
        st.metric("Potential Revenue", f"${prospects['monetary'].sum():,.2f}")
        
        return prospects

# Example usage (this won't run in notebook, but shows how to use in Streamlit)
"""
if __name__ == "__main__":
    st.set_page_config(
        page_title="Donor Analytics Dashboard",
        page_icon="🎯",
        layout="wide"
    )
    
    # Initialize dashboard
    dashboard = DonorDashboard(features_df, model)
    
    # Sidebar navigation
    page = st.sidebar.selectbox(
        "Navigate",
        ["Donor Search", "Geographic View", "RFM Analysis", "Campaign Simulator"]
    )
    
    if page == "Donor Search":
        results = dashboard.donor_search()
        if results is not None:
            selected_donor = st.selectbox(
                "Select Donor",
                results['donor_id'].tolist()
            )
            if selected_donor:
                dashboard.donor_profile(selected_donor)
                
    elif page == "Geographic View":
        dashboard.geographic_view()
        
    elif page == "RFM Analysis":
        dashboard.rfm_analysis()
        
    elif page == "Campaign Simulator":
        prospects = dashboard.campaign_simulator()
        if len(prospects) > 0:
            st.dataframe(prospects)
"""